In [ ]:
import pandas as pd
import requests
import csv
from threading import Timer
import json
import time
import os
import datetime
import pymongo
from pymongo import MongoClient

Scaricamento lista di tutte le stazioni.

In [ ]:
temp = []
stazioni = []
for i in range(65,91):
    r = requests.get("http://www.viaggiatreno.it/infomobilita/resteasy/viaggiatreno/autocompletaStazione/" + chr(i))
    temp.append(r.text.split("\n"))

for i in temp:
    for j in range(0,len(i)-1):
        stazioni.append(i[j])

stripped = (line.strip() for line in stazioni)
lines = (line.split("|") for line in stripped if line)

with open('stazioni.csv',  'w') as out_file:
    writer = csv.writer(out_file)
    writer.writerow(('nome_stazione', 'id'))
    writer.writerows(lines)

Creazione dei dizionari treni_stazioni e treni_partenze (per il giorno seguente).

In [ ]:
data = pd.read_csv("stazioni.csv")

In [ ]:
def crea_dizionari():
    
    df = pd.read_csv('stazioni.csv')
    stazioni = df["id"]
    mese = "Jun"
    anno = 2022
    giorni_settimana = ['Mon','Tue','Wed','Thu','Fri','Sat','Sun','Mon']
    giorno_settimana = giorni_settimana[datetime.datetime.today().weekday()+1]
    treni_stazioni = {}
    treni_partenze = {}
    
    giorno = int(time.strftime("%d")) + 1
    tratta = list()
    count = 0
    for i in stazioni:
        count += 1
        for h in range(1,23,2):
            try:
               r = requests.get("http://www.viaggiatreno.it/infomobilita/resteasy/viaggiatreno/partenze/" 
                                + i + "/" + giorno_settimana +
                                "%20"+ mese +"%20" + str(giorno) +"%20" + str(anno) + "%20" 
                                + str(h) + ":00:00%20GMT+0100")

            except (requests.ConnectionError, requests.Timeout, requests.RequestException) as e:
                print("OOPS!! Connection Error. Make sure you are connected to Internet.\n")
                print("I'm gonna take a break...\n")
                time.sleep(10)
                r = requests.get("http://www.viaggiatreno.it/infomobilita/resteasy/viaggiatreno/partenze/" + i + "/" + giorno_settimana +
                                "%20"+ mese +"%20" + str(giorno) +"%20" + str(anno) + "%20" + str(h) + ":00:00%20GMT+0100")            
          
            except KeyboardInterrupt:
                print("Someone closed the program")
   
            if r.status_code == 200:
                tratta.append(r.json())

        for ora in tratta:
            for treno in ora:
                if treno['codOrigine'] == i:
                    treni_stazioni[treno['numeroTreno']] = treno['codOrigine']
                    treni_partenze[treno['numeroTreno']] = treno['orarioPartenza']
        tratta = list()
        print(count)
    
    with open('treni_stazioni_' + str(datetime.date.today() + datetime.timedelta(days=1)) +'.json', 'w') as outfile:
        json.dump(treni_stazioni, outfile)
    
    with open('treni_partenze_' + str(datetime.date.today() + datetime.timedelta(days=1)) +'.json', 'w') as outfile:
        json.dump(treni_partenze, outfile)

In [ ]:
crea_dizionari()

Scaricamento dati treni.

In [ ]:
client = MongoClient('localhost', 27017)
db = client.dati_viaggiatreno

In [ ]:
#TIMER
x = datetime.datetime.today()
y = x.replace(day=x.day, hour=22, minute=50, second=0, microsecond=0)
delta_t = y-x
secs = delta_t.seconds+1

In [ ]:
def createFolder(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print ('Error: Creating directory. ' +  directory)

In [ ]:
def richieste_viaggiotreno():
    
    print("Start: " + time.strftime("%H:%M:%S"))
    treni_stazioni = json.loads(open("treni_stazioni_" + str(datetime.date.today()) + ".json").read())
    treni_partenze = json.loads(open("treni_partenze_" + str(datetime.date.today()) + ".json").read())

    createFolder(time.strftime("%d-%m-%Y"))
    treni = []
    time.strftime("%d-%m-%Y")
    for i in treni_stazioni.keys():
        treni.append(i)
        
    giorno = db[time.strftime("%d-%m-%Y")]
    
    temp = []
    count = 0
    conn_error = 0
    timeout_error = 0
    gen_error = 0
    
    for i in range(0,len(treni_stazioni)):

        try:
            r = requests.get("http://www.viaggiatreno.it/infomobilita/resteasy/viaggiatreno/andamentoTreno/" 
                             + str(treni_stazioni[treni[i]]) + "/" + str(treni[i]) + "/" + str(treni_partenze[treni[i]]))

        except (requests.ConnectionError, requests.Timeout, requests.RequestException) as e:
            timeout_error += 1
            print("OOPS!! Timeout Error")
            print("I'm gonna take a break...\n")
            time.sleep(10)
            
            print("I'm restarting from station:",treni_stazioni[treni[i]]," - train: ",treni[i],"\n")
            r = requests.get("http://www.viaggiatreno.it/infomobilita/resteasy/viaggiatreno/andamentoTreno/" + str(treni_stazioni[treni[i]]) + "/" + str(treni[i]) + "/" + str(treni_partenze[treni[i]]))

        except KeyboardInterrupt:
            print("Someone closed the program")
    
        if r.status_code == 200:
            temp.append(r.json())
            
            with open(time.strftime("%d-%m-%Y") +'/' + str(treni[i]) + '_' + time.strftime("%d-%m-%Y") + '.json', 'w') as outfile:
                json.dump(temp, outfile)
            x = giorno.insert_many(temp)
            
            temp = list()
            print ("File: ", count, " - station: ", treni_stazioni[treni[i]]," - train: ",treni[i]," - starting time: ",treni_partenze[treni[i]],"\n" )
            count += 1

    print("End:" + time.strftime("%H:%M:%S"))
    print("Connection Error: ", conn_error)
    print("Timeout Error: ", timeout_error)
    print("General Error: ", gen_error)

In [ ]:
t = Timer(secs, richieste_viaggiotreno)
t.start()